# MIRA Regulatory Potential Modeling

In [ ]:
!hostnamectl

In [ ]:
import mira
import scanpy as sc
import anndata
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc("font", size = 12)

In [ ]:
rna_adata = anndata.read_h5ad("/gpfs/Home/esm5360/MIRA/mira-datasets/ds011_rna_data_joint_representation.h5ad")
atac_adata = anndata.read_h5ad("/gpfs/Home/esm5360/MIRA/mira-datasets/ds011_atac_data_joint_representation.h5ad")

ds011_atac_model = mira.topics.load_model("/gpfs/Home/esm5360/MIRA/mira-datasets/ds011_atac_model.pth")
ds011_rna_model = mira.topics.load_model("/gpfs/Home/esm5360/MIRA/mira-datasets/ds011_rna_model.pth")

## TSS Annotations

We must annotate the genes in the dataset with TSS locations. MIRA provides hg38 and mm10 chromosome size and TSS data, but you can also use your own if you want. The format is a bit different for the TSS data than the one that I have been using

In [ ]:
mira.datasets.mm10_chrom_sizes()
mira.datasets.mm10_tss_data()

We need to add `peak_id`, `chr`, `start`, and `end` columns containing peak location information to `atac_adata.var`

In [ ]:
peak_locations = atac_adata.var.index

if not any(["chr", "start", "end"]) in peak_locations:
    peak_data = {
        "peak_id": [],
        "chr": [],
        "start": [],
        "end": []
    }
    for i, peak in enumerate(peak_locations):
        peak_id = i
        chr_num = peak.split(":")[0]
        peak_start = int(peak.split(":")[1].split("-")[0])
        peak_end = int(peak.split(":")[1].split("-")[1])
        
        peak_data["peak_id"].append(peak_id)
        peak_data["chr"].append(chr_num)
        peak_data["start"].append(peak_start)
        peak_data["end"].append(peak_end)
        
    peak_df = pd.DataFrame(peak_data, index=peak_locations)
    atac_adata.var = pd.concat([atac_adata.var, peak_df], axis=1)

In [ ]:
atac_adata

In [ ]:
tss_data_file = 'mira-datasets/mm10_tss_data.bed12'
tss_data = pd.read_csv('mira-datasets/mm10_tss_data.bed12', sep="\t")
tss_data["#geneSymbol"] = tss_data["#geneSymbol"].str.capitalize()
tss_data.to_csv('mira-datasets/mm10_tss_data.bed12', sep="\t", header=True, index=False)
tss_data

In [ ]:
mira.tl.get_distance_to_TSS(atac_adata,
                            tss_data='mira-datasets/mm10_tss_data.bed12',
                            genome_file='mira-datasets/mm10.chrom.sizes')

## RP model training

With TSS annotations added, we can train RP models. Most RP-model related funcitons take the `expr_adata` and `atac_adata` keyword arguments, so it's easiest to put `rna_adata` and `atac_adata` into a dictionary for repeated use:

In [ ]:
rp_args = dict(expr_adata = rna_adata, atac_adata= atac_adata)

Next, we instantiate an RP Model. MIRA refers to the RP model variant that uses local chromatin to predict gene expression as a *LITE* model - or **L**ocal chromatin accessibility-**I**nfluenced **T**ranscriptional **E**xpression model.

The `mira.rp.LITE_Model` object takes the expression and accessibility topic models. You must also define a list of genes to model. To keep this short, I will demonstrate training with four genes, but in a full analysis, we recommend training models for *all highly-variable genes, plus all genes that scored in the top 5% most-activated for any topic*, which gives a good survey of interesting gene expression variation in your data. This following snippet gives you that genelist:

In [ ]:
rp_genes = list(ds011_rna_model.features[ds011_rna_model.highly_variable])
for topic in range(ds011_rna_model.num_topics):
    rp_genes.extend(ds011_rna_model.get_top_genes(topic, 200))
rp_genes = list(set([i.capitalize() for i in rp_genes]))
print(rp_genes)

In [ ]:
litemodel = mira.rp.LITE_Model(expr_model = ds011_rna_model,
                              accessibility_model=ds011_atac_model,
                              genes = rp_genes[0:5]) # Just using the first 5 top genes

Now fit the models. Most LITE_model methods take the `n_workers` parameter, which parallelizes across cores. You can provide the `mira.rp.SaveCallback` to the `fit` function, which will save each model as it is trained. One must simply provide the prefix or directory where RP models are to be saved.

`litemodel.fit` will always use the `.X` attribute, but requires integer counts data. We need to set the `rna_adata.X` values back to the raw counts.

In [ ]:
rna_adata.X = rna_adata.layers["counts"]
rp_args = dict(expr_adata = rna_adata, atac_adata= atac_adata)
rna_adata.X

In [ ]:
litemodel.fit(
    **rp_args,
    n_workers=4,
    callback=mira.rp.SaveCallback('/gpfs/Home/esm5360/MIRA/data/ds011_rpmodels/')
)

To reload LITE models from disk, either instantiate the container object and use the load function with the file prefix given above:

In [ ]:
litemodel = mira.rp.LITE_Model(expr_model = ds011_rna_model,
                              accessibility_model=ds011_atac_model,
                              genes = rp_genes[0:5])

litemodel.load('/gpfs/Home/esm5360/MIRA/data/ds011_rpmodels/')

Or to skip having to provide the gene list, use `mira.rp.LITE_Model.load_dir`. This function will load every model in the given directory.

In [ ]:
litemodel = mira.rp.LITE_Model.load_dir(
    expr_model = ds011_rna_model,
    accessibility_model = ds011_atac_model,
    prefix='/gpfs/Home/esm5360/MIRA/data/ds011_rpmodels/'
)

## Defining local chromatin neighborhoods

If you are interested in the distance of estimated regulatory influence for a certain gene, you can index on the `litemodel` object with a gene name, then use the `parameters_` attribute (distance is decay rate in kilobases):

In [ ]:
first_gene = rp_genes[0]
print(first_gene)
litemodel[first_gene].parameters_

Or access the parameters of all models like so:

In [ ]:
pd.DataFrame(
    litemodel.parameters_
).T

Say one wanted a list of all peaks within the influence of Gpc6’s RP model. You can quickly access the peaks that make up a gene’s local cis-regulatory neighborhood using `get_influential_local_peaks`. This function takes the parameter `decay_periods`, which defines the distance covered by the gene’s chromatin neighborhood in terms of the `decay_periods` times the RP model’s upstream and downstream decay distances.

In [ ]:
litemodel['Gpc6'].get_influential_local_peaks(atac_adata, decay_periods = 5.).head(5)


You can also manually access TSS-peak distances via:

In [ ]:
tss_distances = mira.utils.fetch_gene_TSS_distances(atac_adata)
tss_distances

In this matrix, peaks that are **upstream** of a gene have **negative** distances, while **downstream** have **positive**. All peaks that are outside of some range (by default 600 kilobases) are masked and have “zero” distance. if a peak begins exactly on the TSS of a gene, it is adjusted to be one base pair distant to avoid getting masked.

## Predicting expression from accessibility

With trained RP models, the `predict` function calculates the maximum aposteriori prediction of expression given the accessibility state of each gene in each cell. Additionally, the model quantifies the likelihood of that prediction.

In [ ]:
litemodel.predict(**rp_args)

You can survey the goodness of fit by checking UMAPs. Compare the corresponsdance between accesibility and expression

**LITE Model Predictions:**

In [ ]:
sc.pl.umap(rna_adata, color = litemodel.genes, frameon=False, color_map='viridis',
          layer='LITE_prediction', ncols=3, vmin = 0, vmax = 'p97')

**Gene Expression:**

In [ ]:
sc.pl.umap(rna_adata, color = litemodel.genes, **mira.pref.raw_umap(ncols=3, size = 20))

## Gene-TF targeting

### With probabilistic in-silico deletion (pISD)

RP models define a local chromatin neighborhood where changes in accessibility appear to influence gene expression. One may assume that transcription factor binding in many cis-regulatory elements within this neighborhood suggests the transcription factor regulates the gene of interest. MIRA can query for these types of interactions at a systems level, finding potential regulatory associations across many gene-TF pairs.

The algorithm for calculating these associations is called probabilistic *in-silico* deletion (pISD), and it measures the ability of the RP model to predict expression of a gene before and after the regulatory elements predicted to bind a certain transcription factor are masked. In this way, pISD simulates a “computational knock out” of that factor.

To compute association scores using each RP model against motif-based binding site predictions of all available transcription factors, use `litemodel.probabilistic_isd`:


In [ ]:
litemodel.probabilistic_isd(**rp_args, n_workers = 4)